# Day 17: Trick Shot
* Peak height seems like an easy calculation as the upward velocity just drops by 1 each step and once we hit zero it wont go higher.
* Part 1 only cares about Peak Y that'll still hit so I really don't even need to care about X at this time.
* The hit tests is a little more involved but seems also pretty straight forward
    * Basically we know that X will trend toward 0 and then settle there from either direction, so we can easily calculate if X will ever make it to our target and if it does, how many steps it'll take. For now we'll assume our X velocity will always be positive.
    * For Y, our gravity constant is 1 which makes this easier. In other words we don't even need to check steps per say as Y will always hit 0 again before going negative. I.E, if our initial velocity is 2, our velocity over time will be 2 -> 1 -> 0 -> -1 -> -2 -> -3 -> -4 etc. This means our points after origin will be 2 -> 3 -> 3 -> 2 -> 0 -> -3 -> -7 etc. In this case it'd take 5 steps to be back to y=0 and then start dropping.
        * If initial velocity is 3 then we get 3 -> 2 -> 1 -> 0 -> -1 -> -2 -> -3 -> -4, and 3 -> 5 -> 6 -> 6 -> 5 -> 3 -> 0 -> -4. That's 7 steps til touching origin again and our velocity from there is essentially -(1+v). 
        * Thought it may not matter, we can calculate the steps lost in upward velocity before we hit origin again by doing 2v+1
        * So this is an easier calculation as we know our upward cancels in proportion to our start and from then provides us with an initial downward velocity to begin calculating for target breach. Which then simply becomes the same exercise as what we do for X

In [ ]:
let inline stepsAboveOrigin iVel = (2*iVel)+1
let inline getUpwardYSteps initialV = Array.unfold (fun (y,v) -> if v = 0 then None else Some (y + v, (y + v, v - 1))) (0, -initialV - 1)
let inline getLowerYSteps initialV targetY = Array.unfold (fun (y,v) -> match y + v with i when i < targetY -> None | i -> Some (i, (i, v - 1))) (0, -initialV - 1)
let inline getXSteps targetX = Array.unfold (fun (x,v) -> match x - v with i when i <= 0 -> None | i -> Some (i, (i, v + 1))) (targetX, 0)
let inline getLeastX targetX = getXSteps targetX |> Array.length
let inline countYSteps initialV lowerY = (stepsAboveOrigin initialV) + (getLowerYSteps initialV lowerY |> Array.length)

let inline calculatePeakY targetLowerBound = getUpwardYSteps targetLowerBound |> Array.last

//  Sample
calculatePeakY -10 
//  Input
calculatePeakY -117


6786

In [ ]:
// We already know our initial Y can't go mor than -TargetLowerBound - 1 otherwise it'll miss entirely
let (sampleYL, sampleYU) = -10,-5
// For our sample we know that this means that the highest initial Y we can support is -(-10) - 1 or 9
// Our lowest Y san be assumed to be a single step or in other words our lowest Y Bound

// We can further elaborate for later by aligning these with the number of steps it takes to reach each
// (Steps, Initial Velocity, Target Y)
//[|-sampleYL - 1 .. -1 .. sampleYL|] |> Array.map (fun y -> (countYSteps y sampleYL, y, getLowerYSteps y -10 |> Array.last)) |> display |> ignore

let (sampleXL, sampleXR) = 20,30
// Our x velocity changes by 1 with every step trending toward 0 where it'll rest. Therefore, we can calulate the slowest possible X to get us to target
//  working backwards from our leftmost X. In our Sample that's 20 and we'll increase our vel by 1 till we hit 0.
//      Vel 0 -> 1 -> 2 -> 3 -> 4; 20 -> 19 -> 17 -> 14 -> 10 -> 5 -> -1. Adding up the steps, 6 is the least we can go before we have not enough energy 
//      to ever get there.
//      0+5=5;5+4=9;9+3=12;12+2=14;14+1=15;15+0=stuck
//      0+6=6;6+5=11;11+4=15;15+3=18;18+2=20 hit
// Similarly we can assume that our X can be bounded by one step, otherwise or rightmost X value
[|20..30|] |> Array.map (fun i -> 
    let stepBack = getXSteps i
    stepBack |> display |> ignore
    let initialVelocity = Array.length stepBack
    let steps = Array.unfold (fun (x,v) -> if v = 0 then None else match x + v with i when i <= 30 -> Some (i, (i, v - 1)) | _ -> None) (0, initialVelocity)
    (Array.length steps, initialVelocity, Array.last steps)
)//|> (fun a -> (Array.length a, Array.filter (fun f -> f >= 20 && f <= 30) a), (Array.last a) + 1)) 
|> display |> ignore 
// [| getLeastX sampleXL..sampleXR |] //|> Array.map (fun x -> (getLowerYSteps))

// [|for x in xCandidates do for y in yCandidates do (x,y)|]

In [ ]:
Array.unfold (fun (x,v) -> if v = 0 then None else match x + v with i when i <= 30 -> Some (i, (i, v - 1)) | _ -> None) (0, 6)

index,value
0,6
1,11
2,15
3,18
4,20
5,21


In [ ]:
let inline getLowerYSteps initialV targetY = Array.unfold (fun (x,v))

let getPeak ((x1:int,x2:int),(y1,y2)) (vx:int, vy:int) =
    //  Check if our X velocity will ever be enough to get us to target
    let xSteps = Seq.unfold (fun (x,v) -> if v = 0 then None else match x + v with i when i > x2 -> None | i -> Some (i, (i, v - 1))) (0,vx)
    if Seq.isEmpty xSteps then None
    elif (Seq.last xSteps) < x1 then None
    //  X trends toward 0 and stops so if we don't fall short we'll likely hit it
    //  There is the posibility that our initial X velocity will overshoot the target on step 1, but we'll avoid that by not checking velocities
    //      that could put us in that situation and thus simplifying our calculation
    else
        // On to calculating Y now
        let startingDownwardVelocity = -(1+vy)
        // So let's calculate all the points between our downward vel and our target to see if we get there
        let ySteps = Seq.unfold (fun (y,v) -> match y + v with i when i < y1 -> None | i -> Some (i, (i, v - 1))) (0, startingDownwardVelocity)
        if Seq.isEmpty ySteps then None
        elif (Seq.last ySteps) > y2 then None
        // If we got this far, then we'll land within our target so let's calculate our peak
        else 
            let peaks = Seq.unfold (fun (y,v) -> if v = 0 then None else Some (y + v, (y + v,v - 1))) (0, vy)
            if Seq.isEmpty peaks then None else Seq.last peaks |> Some
        
let findCandidates (x1:int,x2:int) (y1:int,y2:int) =
    //  We know that any X velocity beyond our furthest target wont do any good so skip that
    //  Same goes for our Y velocity but that is translated so we'll skip any -(v+1) velocities beyond our lowest point

    [|
        for x = 0 to x2 do
            for y = 0 downto y1 + 1 do
                getPeak ((x1,x2),(y1,y1)) (x,y)
    |]

findCandidates (20,30) (-10,-5)

Error: System.OperationCanceledException: Command :SubmitCode: let getPeak ((x1:int,x2:int),(y1,y2)) (vx:int, vy: ... cancelled.